<a href="https://colab.research.google.com/github/Ubaid-Manzoor/Automatic-Serveillance-using-ML/blob/master/AutomaticServaliance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOTE:- THIS CODE WAS RUNNED ON GOOGLE COLAB, THERE ARE THING WHICH WONT RUN IN SOME COMPUTER

# Import all Libraries

In [22]:
import os
import cv2
import copy
import time
import math
import torch
import torchvision
import numpy as np
import torch.nn as nn
from PIL import Image
import torch.optim as optiom
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms 
from torch.utils.data import Dataset, IterableDataset, DataLoader

use_gpu = torch.cuda.is_available()
if use_gpu:
  print("Using gpu")

Using gpu


# Downloading the data

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [136]:
########################################################################
########################################################################
###################### SOME USEFUL FUNCTIONS ###########################
########################################################################
########################################################################
import zipfile

def removedir(path):
  """Remove all file and directory recursively"""
  if(os.path.isfile(path)):
    os.remove(path)
  elif( os.path.isdir(path) ):
    path_list = os.listdir(path)
    for path_ in path_list:
      removedir(os.path.join(path,path_) )
    os.rmdir(path)

def unzip(path, dest):
  with zipfile.ZipFile(path, 'r') as file_to_zip:
    file_to_zip.extractall(dest)


########################################################################
########################################################################
########################################################################

In [ ]:
#################### Download and Unziping the Dataset ############
from torch.hub import download_url_to_file

url = 'https://uca99ec3038c3fcdd6df2668dc1b.dl.dropboxusercontent.com/cd/0/get/A5ZKwNLPDmodhwYEmwu_IQRSkPJQs9qKvoxQQ5lhqKe9TjoTZ_o1TTFD7n2Cv3Qn4EC5Jd9v_9-ZXhLxIgLh1Kur7dnQedFvdBtUQvnkeAwJoQ/file?_download_id=8495528592147592612394157372631981580157083575271750206206115965&_notify_domain=www.dropbox.com&dl=1'
path = "/content/drive/My Drive/AutomaticServaliance Project/Data"
download_url_to_file(url,path)

# file = "/content/drive/My Drive/AutomaticServaliance Project/"
# unzip(file,path)

In [ ]:
file = "/content/drive/My Drive/AutomaticServaliance Project/Data/tmprv1i177x"
unzip(file,path)

# Transforming the Data

In [ ]:
TRAIN = 'train'
VAL = 'val'
TEST = 'test'


data_transforms = {
    TRAIN: transforms.Compose([
                               transforms.ToTensor()
    ]),
    VAL: transforms.Compose([
                             transforms.ToTensor()
    ]),
    TEST: transforms.Compose([
                              transforms.ToTensor()
    ])
}



In [ ]:
class VideoDataset(IterableDataset):
  def __init__(self,path,type='train'):
    self.path = path 
    self.train_file = path + '/' + 'train.txt'
    self.validation_file = path + '/' + 'validation.txt'
    self.type = type
    self.train_per = 70
  

  def pathLists(self,path):
    """ Return List of All the paths(Full) of the videos """
    pathlist = []

    for dirname in os.listdir(path):
      if os.path.isdir(path + '/' + dirname):
        full_path = [path + '/' + dirname + '/' + filename for filename in os.listdir(path + '/' + dirname)]
        pathlist.append(full_path)
    return np.array(pathlist)

  def read_video(self,path):
    vid_cap = cv2.VideoCapture(path)
    success, frame = vid_cap.read()
    video_tensor = torch.tensor(frame).unsqueeze(3)

    count = 0
    while success:
      success,frame = vid_cap.read()
      if success and count%10 == 0:
        video_tensor = torch.cat((video_tensor,torch.tensor(frame).unsqueeze(3)),3)
      count += 1
    return video_tensor

  def parse_file(self):
    """ Yeild a Video from the data """
    if self.type == 'train':
      if not os.path.exists(self.train_file):
        with open(self.train_file,'w') as file:
          for fileList in self.pathLists(path):
            for line in fileList[0:math.floor((len(fileList)/100) * self.train_per)]:
              file.writelines(line+'\n')
      if os.path.exists(self.train_file) and os.stat(self.train_file).st_size > 0:
        with open(self.train_file) as file:
          for video_path in file:
            yield self.read_video(video_path[:-1])

    elif self.type == 'validation': 
      if not os.path.exists(self.validation_file):
        with open(self.validation_file,'w') as file:
          for fileList in self.pathLists(path):
            for line in fileList[math.floor((len(fileList)/100) * self.train_per):]:
              file.writelines(line+'\n')  
      if os.path.exists(self.validation_file) and os.stat(self.validation_file).st_size > 0:
        with open(self.validation_file) as file:
          for video_path in file:
            yield self.read_video(video_path[:-1])
       
  def __iter__(self):
    return iter(self.parse_file())





## Convert Frame Into Features

In [73]:
def load_model(model_type="vgg16"):
  if model_type == "vgg16":
    model = models.vgg16(pretrained=True,progress=True)
    return model.features

def preprocess_frame(frame):
  frame = np.reshape(frame,(frame.shape[2],frame.shape[0],frame.shape[1]))
  preprocess = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
  ])

  return preprocess(transforms.functional.to_pil_image(frame)).unsqueeze(0)

  

def get_features(frame):
  vgg_model = load_model()
  processed_frame = preprocess_frame(frame)
  vgg_model.eval()
  features = vgg_model(processed_frame)
  return torch.flatten(features)


def save_features(features, path_to_save, frame_no):
  torch.save(features,os.path.join(path_to_save,str(frame_no)+".pt"))

def get_frame_to_take(total_frames,frames_to_take=500):
  frames_taken = np.array([])

  if total_frames < frames_to_take:
    return np.array(list(range(total_frames)))

  for i in range(0,total_frames):
    if(i%int(total_frames/frames_to_take) == 0):
      frames_taken = np.append(frames_taken,i)

  
  frames_to_remove = np.array([])
  frames_to_remove = frames_to_remove.astype('int32')
  
  for i in range(len(frames_taken)):
    if len(frames_to_remove) == (len(frames_taken) - frames_to_take):
      break
    if i%2 == 0:
      frames_to_remove = np.append(frames_to_remove,i)
    if len(frames_to_remove) == (len(frames_taken) - frames_to_take):
      break
    if (len(frames_taken) - i)%2 == 0:
      frames_to_remove = np.append(frames_to_remove,len(frames_taken) - i - 1)
  
  return set(np.delete(frames_taken,frames_to_remove))

def yeildframe(videopath):
  video = cv2.VideoCapture(videopath)

  frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  frames_to_take = get_frame_to_take(frame_count)
  
  count = 0;success = True;first=True
  while success:
    success,frame = video.read()
    if success and count in frames_to_take:
      yield frame
    count += 1


def filterfolders(paths,folder_to_exclude=[]):
  return [path for path in paths \
                    if os.path.isdir(path) \
                        and \
                      path.split('/')[-1] not in folder_to_exclude ]

def filterfiles(paths):
  return [path for path in paths \
                    if os.path.isfile(path)]

def getfullpath(base_path,folders):
  return [os.path.join(base_path, folder) for folder in folders]

def convert_videoFrame_to_feature(path):
  datafolder = filterfolders(getfullpath(path,os.listdir(path)),["train",".ipynb_checkpoints"])
  os.makedirs(os.path.join(path,"features"),exist_ok=True)
  
  for classfolder in datafolder:
    class_name = classfolder.split('/')[-1]
    os.makedirs(os.path.join(path,"features",class_name),exist_ok=True)
    videopaths = getfullpath(classfolder,os.listdir(classfolder))
    
    for video in videopaths:
      video_name = video.split('/')[-1]
      os.makedirs(os.path.join(path,"features", class_name,video_name),exist_ok=True)

      for frame_no,frame in  enumerate(yeildframe(video)):
        frame_features = get_features(frame)
        save_features(frame_features,os.path.join(path,"features", class_name,video_name),frame_no)
      print(video.split('/')[-1], " Compeleted!!!")

In [ ]:
path = '/content/drive/My Drive/AutomaticServaliance Project/Data/Anomaly-Videos-Part-1'
convert_videoFrame_to_feature(path)

Abuse041_x264.mp4  Compeleted!!!
Abuse037_x264.mp4  Compeleted!!!
Abuse012_x264.mp4  Compeleted!!!
Abuse001_x264.mp4  Compeleted!!!
Abuse048_x264.mp4  Compeleted!!!
Abuse003_x264.mp4  Compeleted!!!
Abuse006_x264.mp4  Compeleted!!!
Abuse020_x264.mp4  Compeleted!!!
Abuse018_x264.mp4  Compeleted!!!
Abuse025_x264.mp4  Compeleted!!!
Abuse029_x264.mp4  Compeleted!!!
Abuse042_x264.mp4  Compeleted!!!
Abuse032_x264.mp4  Compeleted!!!
Abuse045_x264.mp4  Compeleted!!!
Abuse049_x264.mp4  Compeleted!!!
Abuse026_x264.mp4  Compeleted!!!
Abuse044_x264.mp4  Compeleted!!!
Abuse031_x264.mp4  Compeleted!!!
Abuse005_x264.mp4  Compeleted!!!
Abuse015_x264.mp4  Compeleted!!!
Abuse038_x264.mp4  Compeleted!!!
Abuse033_x264.mp4  Compeleted!!!


In [ ]:
### If This dataset is use we cant use Shuffle


class ImageDataset(IterableDataset):
  def __init__(self,path,type='train'):
    self.path = path 
    self.train_file = path + '/' + 'train.txt'
    self.validation_file = path + '/' + 'validation.txt'
    self.type = type
    self.train_per = 70
  

  def pathLists(self,path):
    """ Return List of All the paths(Full) of the videos """
    pathlist = []

    for dirname in os.listdir(path):
      if os.path.isdir(path + '/' + dirname):
        full_path = [path + '/' + dirname + '/' + filename for filename in os.listdir(path + '/' + dirname)]
        pathlist.append(full_path)
    return np.array(pathlist)

  def read_video(self,path):
    vid_cap = cv2.VideoCapture(path)
    success, frame = vid_cap.read()

    count = 0
    while success:
      if success and count%10 == 0:
        yield torch.tensor(frame)
      success,frame = vid_cap.read()
      count += 1

  def parse_file(self):
    """ Yeild a Video from the data """
    if self.type == 'train':
      if not os.path.exists(self.train_file):
        with open(self.train_file,'w') as file:
          for fileList in self.pathLists(path):
            for line in fileList[0:math.floor((len(fileList)/100) * self.train_per)]:
              file.writelines(line+'\n')
      if os.path.exists(self.train_file) and os.stat(self.train_file).st_size > 0:
        with open(self.train_file) as file:
          for video_path in file:
            for image in self.read_video(video_path[:-1]):
              yield image

    elif self.type == 'validation': 
      if not os.path.exists(self.validation_file):
        with open(self.validation_file,'w') as file:
          for fileList in self.pathLists(path):
            for line in fileList[math.floor((len(fileList)/100) * self.train_per):]:
              file.writelines(line+'\n')  
      # if os.path.exists(self.validation_file) and os.stat(self.validation_file).st_size > 0:
      #   with open(self.validation_file) as file:
      #     for video_path in file:
      #       for image in read_video(video_path[:-1]):
      #         yield image
       
  def __iter__(self):
    return iter(self.parse_file())

In [ ]:
path = '/content/drive/My Drive/AutomaticServaliance Project/Data/Anomaly-Videos-Part-1'
dataset = ImageDataset(path,type='validation')
dataloader = DataLoader(dataset,batch_size=10)

In [ ]:
for data in dataloader:
  # print(data.shape)
  for image in data:
    plt.imshow(image)
  break

## Need to extract Image from the Video and Save them according to their Classes

In [ ]:
def extract_video(path, start_per=15, end_per=85,stride=10):
  video = cv2.VideoCapture(path)

  frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  frame_rate = int(video.get(cv2.CAP_PROP_FPS))

  if( (frame_count/frame_rate)/60 > 5 ):
    stride = 20
  elif( (frame_count/frame_rate)/60 > 10 ):
    stride = 30
  starting_frame = math.floor((frame_count/100) * start_per)
  ending_frame = math.floor((frame_count)/100 * end_per)

  print("total frames : ",frame_count)
  print("total image : ",(ending_frame - starting_frame) / stride )
  frame_num = 0
  success = True
  while success:
    success,frame = video.read()
    frame_num += 1
    if success and (frame_num%stride) == 0:
      yield np.array(frame)

def extract_and_save_Images(paths_file, path_to_store="",folder_name="",extension='.jpg'):
  try:
    with open(paths_file) as file:
      paths = file.readlines()
      image_number = 0
      for video_path in paths:
        image_class = video_path.split('/')[-2]
        folder_path = os.path.join(path_to_store , folder_name , image_class )
        os.makedirs(folder_path, exist_ok=True)

        for image in extract_video(video_path[:-1]):
          try:
            cv2.imwrite(os.path.join(folder_path,image_class + '_' + str(image_number) + extension),image)
            image_number += 1
          except FileNotFoundError:
            print(f"{path_to_store} does not exists")
  except FileNotFoundError:
    print(f"{paths_file} does not exists")

In [ ]:
paths_file = "/content/drive/My Drive/AutomaticServaliance Project/Data/Anomaly-Videos-Part-1/train.txt"
extract_and_save_Images(paths_file,os.path.abspath(os.path.join(paths_file,'..')),"train")

In [ ]:
path = "/content/drive/My Drive/AutomaticServaliance Project/Data/Anomaly-Videos-Part-1/train/Assault"
print(len(os.listdir(path)))

8331
